In [3]:
import requests

import numpy as np
import pandas as pd
import requests
import acquire

ModuleNotFoundError: No module named 'acquire'

In [2]:
# https://aphorisms.glitch.me returns a random quotation
response = requests.get('http://aphorisms.glitch.me/')
response


<Response [200]>

In [3]:
response.ok


True

In [4]:
response.ok


True

In [5]:
response.text


'{"author":"Johannes Brahms","quote":"Without craftsmanship, inspiration is a mere reed shaken in the wind."}'

In [6]:
url = 'https://swapi.dev/api/people/5'
response = requests.get(url)
print(response.text)



{"name":"Leia Organa","height":"150","mass":"49","hair_color":"brown","skin_color":"light","eye_color":"brown","birth_year":"19BBY","gender":"female","homeworld":"https://swapi.dev/api/planets/2/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/30/"],"starships":[],"created":"2014-12-10T15:20:09.791000Z","edited":"2014-12-20T21:17:50.315000Z","url":"https://swapi.dev/api/people/5/"}


In [7]:
data = response.json()
print(type(data))
data




<class 'dict'>


{'name': 'Leia Organa',
 'height': '150',
 'mass': '49',
 'hair_color': 'brown',
 'skin_color': 'light',
 'eye_color': 'brown',
 'birth_year': '19BBY',
 'gender': 'female',
 'homeworld': 'https://swapi.dev/api/planets/2/',
 'films': ['https://swapi.dev/api/films/1/',
  'https://swapi.dev/api/films/2/',
  'https://swapi.dev/api/films/3/',
  'https://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['https://swapi.dev/api/vehicles/30/'],
 'starships': [],
 'created': '2014-12-10T15:20:09.791000Z',
 'edited': '2014-12-20T21:17:50.315000Z',
 'url': 'https://swapi.dev/api/people/5/'}

In [8]:
base_url = 'https://python.zgulde.net'
print(requests.get(base_url).text)


{"api":"/api/v1","help":"/documentation"}



In [9]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])



The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [10]:
response = requests.get('https://python.zgulde.net/api/v1/items')

data = response.json()
data.keys()


dict_keys(['payload', 'status'])

In [11]:
data['payload'].keys()


dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [12]:
current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')


current_page: 1
max_page: 3
next_page: /api/v1/items?page=2


In [13]:
data['payload']['items'][:2]


[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

In [14]:
import pandas as pd

df = pd.DataFrame(data['payload']['items'])
df.head()


,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [15]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()


current_page: 2
max_page: 3
next_page: /api/v1/items?page=3


In [16]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()


current_page: 3
max_page: 3
next_page: None


In [17]:
df.shape


(50, 8)

1)Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [4]:
def get_items(url='https://python.zgulde.net/api/v1/items?page=1'):
    max_page = requests.get(url).json()['payload']['max_page'] + 1
    for i in range(1,max_page):
        url = url[:-1] + str(i)
        if i == 1:
            output = pd.DataFrame(requests.get(url).json()['payload']['items'])
        else:
            output = pd.concat([output, pd.DataFrame(requests.get(url).json()['payload']['items'])], 
                               ignore_index=True)
    return output


In [5]:
items = get_items()
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


2)Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [6]:
def get_stores(url='https://python.zgulde.net/api/v1/stores'):
    return pd.DataFrame(requests.get(url).json()['payload']['stores'])

In [7]:
get_stores()


,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


3)Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [8]:
def get_sales(url = 'https://python.zgulde.net/api/v1/sales?page='):
    max_page = requests.get(url+'1').json()['payload']['max_page'] + 1
    for i in range(1,max_page):
        new_url = url + str(i)
        if i == 1:
            output = pd.DataFrame(requests.get(new_url).json()['payload']['sales'])
        else:
            output = pd.concat([output, pd.DataFrame(requests.get(new_url).json()['payload']['sales'])], 
                               ignore_index=True)
    return output


In [9]:
sales = get_sales()


4)Save the data in your files to local csv files so that it will be faster to access in the future.

In [10]:
sales.to_csv('sales.csv')


5)Combine the data from your three separate dataframes into one large dataframe.

In [11]:
both = sales.merge(items, left_on='item', right_on='item_id')
database = both.merge(stores, left_on='store', right_on='store_id')


NameError: name 'stores' is not defined

In [ ]:
def combined(sales, stores, items):
    both = sales.merge(items, left_on='item', right_on='item_id')
    return both.merge(stores, left_on='store', right_on='store_id')


In [ ]:
combined(sales, stores, items)


6)Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [ ]:
url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'


In [ ]:
e_data = pd.read_csv(url)


In [ ]:
e_data


7)Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.